In [1]:
!pip install tensorflow fastapi uvicorn pillow numpy opencv-python matplotlib


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 2.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 2.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 2.6 MB/s eta 0:00:01
   ------------------------------------- -- 1.8/2.0 MB 986.7 kB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 888.2 kB/s eta 0:00:00


In [2]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:

# Paths
dataset_path = "./GTSRB/Final_Training/Images"
IMG_HEIGHT, IMG_WIDTH = 64, 64  # Resize images to 64x64 for model
NUM_CLASSES = 43  # GTSRB has 43 traffic sign classes

# Function to load .ppm images and labels
def load_data(data_dir):
    images = []
    labels = []
    
    for class_id in range(NUM_CLASSES):
        class_path = os.path.join(data_dir, f"{class_id:05d}")
        if not os.path.exists(class_path):
            continue
        
        for img_name in os.listdir(class_path):
            if img_name.endswith(".ppm"):  # Load only PPM images
                img_path = os.path.join(class_path, img_name)
                img = Image.open(img_path)  # Open PPM image
                img = img.resize((IMG_WIDTH, IMG_HEIGHT))  # Resize to 64x64
                img = np.array(img) / 255.0  # Normalize pixel values
                
                images.append(img)
                labels.append(class_id)

    return np.array(images), np.array(labels)

# Load training data
X_train, y_train = load_data(dataset_path)

# Convert labels to categorical format (one-hot encoding)
y_train = to_categorical(y_train, NUM_CLASSES)

# Define the model (simple CNN for demonstration)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

# Save the model
model.save("gtsrb_model1.h5")


c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 48s 46ms/step - accuracy: 0.3593 - loss: 2.2726 - val_accuracy: 0.0000e+00 - val_loss: 15.6907
Epoch 2/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - accuracy: 0.7958 - loss: 0.6228 - val_accuracy: 0.0000e+00 - val_loss: 21.1076
Epoch 3/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - accuracy: 0.8622 - loss: 0.4202 - val_accuracy: 0.0000e+00 - val_loss: 23.9866
Epoch 4/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 44s 44ms/step - accuracy: 0.8904 - loss: 0.3231 - val_accuracy: 1.2752e-04 - val_loss: 23.2261
Epoch 5/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - accuracy: 0.9099 - loss: 0.2639 - val_accuracy: 1.2752e-04 - val_loss: 29.0054
Epoch 6/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - accuracy: 0.9242 - loss: 0.2210 - val_accuracy: 2.5504e-04 - val_loss: 26.6650
Epoch 7/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - accuracy: 0.9381 - loss: 0.1842 - val_accuracy: 2.5504e-04 - val_loss: 29.1870
Epoch 8/100
981/981 ━━━━━━━━━━━━━━━━━━━━ 43s 44m

In [8]:
import numpy as np
import tensorflow as tf
from PIL import Image
import json

# Load the trained model
model = tf.keras.models.load_model("gtsrb_model.h5")

# Load class labels from JSON file
with open("class_label.json", "r") as file:
    class_labels = json.load(file)

# Function to preprocess PNG image
def preprocess_image(image_path):
    IMG_HEIGHT, IMG_WIDTH = 64, 64  # Same size as during training
    img = Image.open(image_path).convert("RGB")  # Convert PNG to RGB if it has an alpha channel
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))  # Resize to 64x64
    img = np.array(img) / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension for prediction
    return img

# Function to predict road sign
def predict_road_sign(image_path):
    img = preprocess_image(image_path)  # Preprocess the input image
    predictions = model.predict(img)  # Get predictions from model
    predicted_class = np.argmax(predictions)  # Get the class ID with the highest probability
    return class_labels[str(predicted_class)]  # Return the road sign name based on the class ID

# Example: Predict the road sign from a PNG image
uploaded_image = "E:/FS Project/uploads/00000.png"  # Replace with the path to your uploaded PNG image
result = predict_road_sign(uploaded_image)
print(f"Predicted Road Sign: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
Predicted Road Sign: Vehicles Over 3.5 Tons Prohibited
